In [1]:
import ee
ee.Initialize(project='pyregence-ee')
import datetime
import scripts.analysis_functions as af
import scripts.utils as utils
%load_ext autoreload
%autoreload 2

### Run BS Mapper for a particular year

In [21]:
#testing python / GEE. To be deleted.
# fires_raw = ee.FeatureCollection('projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2020_20230406')
# print(fires_raw.size().getInfo())

# should_zero = fires_raw.filter(ee.Filter.notNull(['nonesuch']))
# print(should_zero.size().getInfo())

# fires_raw.set('this_property', 'something')
# print(fires_raw.)
# should_all = fires_raw.filter(ee.Filter.notNull(['this_property']))
# print(should_all.size().getInfo())

# nonesuch = fires_raw.get('nonesuch')
# print(nonesuch)

# if nonesuch is None:
#     print('yup, not here')

#print(datetime.now())
#print(datetime.datetime.now())

now = ee.Date(datetime.datetime.now())
then = ee.Date(datetime.datetime(2023, 8, 31))

print(now.difference(then, 'days').getInfo())
print(then.difference(now, 'days').getInfo())


7.7966870833333335
-7.7966870833333335


In [27]:
#
#
# TODO

##### IMPORT DATA #####
# One set of yearly fires, no need for remove_recent() or year/ac filtering, that was already done upon data fetch

#***
# USER: Pick either NICF or MTBS import variables, select year that matches with data import (manual), change out folder if needed
#***

yr = '2020' #align with fires import below

# ### OPTION 1: NIFC option
# fires_asset = f"projects/pyregence-ee/assets/conus/nifc/nifc_fires_{yr}_gte100ac_20230406"
# fires = ee.FeatureCollection(fires_asset)
# data_origin = 'nifc'
# asset_folder = 'projects/pyregence-ee/assets/conus/nifc'

# ### OPTION 2: MTBS option
# fires = ee.FeatureCollection("projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2020_20230406")
# data_origin = 'mtbs'
# asset_folder = 'projects/pyregence-ee/assets/conus/mtbs'

### OPTION 3: Test / development case, limited numbers of fires
fires_raw = ee.FeatureCollection('projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2020_20230406')
irwin_list = ['56C3E0AE-98EF-4433-863D-23A5DFF3D0C2', '60146AC0-3AC8-462B-AC80-60D31F1B4D96', 
              '4CE00061-B897-49DC-8839-D08E040FF65D', 'BDE833A4-B278-4418-8526-67F2BFC7D7CD'] 
    #BOBCAT = 56C3E0AE-98EF-4433-863D-23A5DFF3D0C2, DOME = 60146AC0-3AC8-462B-AC80-60D31F1B4D96, 
    #CASTLE = 4CE00061-B897-49DC-8839-D08E040FF65D (note multipolygon), SILVERADO = BDE833A4-B278-4418-8526-67F2BFC7D7CD (late fire 2020-10-26, should be filtered out)
fires = fires_raw.filter(ee.Filter.inList('irwinID',irwin_list))
data_origin = 'mtbs'
#output folder
asset_folder = 'projects/pyregence-ee/assets/fires_bs_tool/dmn_testing'

# Metadata collection and print
fires_count = fires.size().getInfo()
print(f'Total Fires in FeatureCollection: {fires_count}')

##### SET (SIMULATED) RUN DATE #####

sim_date_list = [datetime.datetime(2020, 9, 30), 
                 datetime.datetime(2020, 12, 31), 
                 datetime.datetime(2021, 3, 31), 
                 datetime.datetime(2021, 6, 30)]

sim_date = sim_date_list[0] #possibly will make a loop, setting manual for now
sim_date_ee = ee.Date(sim_date)

# Map over the fires, adding the simulated run date 
#  note: must use subfunction set_windows_sim() in bs calc function
def set_sim_date(feat: ee.Feature):
    #will also need to know if fire was after simulation date
    fire_date = ee.Date(feat.getString('Discovery'))
    #negative days mean Discovery was BEFORE simulated date
    days_before_sim = fire_date.difference(sim_date_ee,'day')
    return feat.set('run_date',sim_date_ee, 'days_before_sim',days_before_sim)
fires_dt = fires.map(set_sim_date)
# Filter fires to only those that were discovered PRIOR to (negative days) the simulated run date
fires_sim = fires_dt.filter(ee.Filter.lt('days_before_sim', 0))

# Metadata collection and print
fires_sim_count = fires_sim.size().getInfo()
print(f'Fire count post simulated date filtering: {fires_sim_count}')


###### METADATA COLLECTION ######
# capture the code run date (today's / current date) as well
cur_dt = datetime.datetime.now()

metadata = {
    'simulated_run_date':sim_date.strftime('%Y%m%d'),
    'code_run_date':cur_dt.strftime('%Y%m%d'),
    'fire_count_original':fires_count,
    'fire_count_final':fires_sim_count,
    'code_origin':'burn-severity-gee::BS_Mapper_testing202308',
}
#print(metadata)

##### RUN BURN SEVERITY #####

# Returns RdNBR & BS for each fire ee.Feature
bs_coll_combined = ee.FeatureCollection(fires_sim.map(af.bs_calc_v2309)) 
bs_coll_combined = ee.ImageCollection(bs_coll_combined)
#print(bs_coll_combined.first().bandNames().getInfo())

# Separate out RdNBR and Miller Threshold severity classes
bs_rdnbr = bs_coll_combined.select('RdNBR')
bs_coll = bs_coll_combined.select('MillersThresholds')

#print(bs_rdnbr.first().bandNames().getInfo())
#print(bs_coll.first().bandNames().getInfo())


# Export intermediate: individual features, RdNBR
# Turn each fire into bands of 1 image to export
# Get irwinID to name bands
fire_name_list = bs_rdnbr.aggregate_array('irwinID')

print(fire_name_list.getInfo())

rdnbr_banded = bs_rdnbr.toBands().rename(fire_name_list)

print(rdnbr_banded.first().bandNames().getInfo())

# # Add metadata
# rdnbr_banded.set(metadata)
# #set file name description with selected variables
# desc_rdnbr = f'bssimrun{yr}_{sim_date.strftime("%Y%m%d")}_{data_origin}_rdnbrbands_{cur_dt.strftime("%Y%m%d")}'
# # Export Image Collection

# utils.exportImgtoAsset(rdnbr_banded, 
#                     desc=desc_rdnbr,
#                     region=None,
#                     asset_folder=asset_folder, 
#                     export_type='single_fire',
#                     export=True)


# # Export intermediate: individual features, severity classified
# # Turn each fire into bands of 1 image to export
# # Get irwinID to name bands
# fire_name_list = bs_coll.aggregate_array('irwinID')
# bs_banded = bs_coll.toBands().rename(fire_name_list)
# # Add metadata
# bs_banded.set(metadata)
# #set file name description with selected variables
# desc_band = f'bssimrun{yr}_{sim_date.strftime("%Y%m%d")}_{data_origin}_bsbands_{cur_dt.strftime("%Y%m%d")}'
# # Export Image Collection

# utils.exportImgtoAsset(bs_banded, 
#                     desc=desc_band,
#                     region=None,
#                     asset_folder=asset_folder, 
#                     export_type='single_fire',
#                     export=True)

# ##### COMPOSITE BURN SEVERITY & FINAL EXPORT #####

# # composite that ee.ImageCollection with a max() reducer
# bs_composite = bs_coll.max().rename('SEVERITY').set('Year',int(yr)) 

# # Add metadata
# bs_composite.set(metadata)

# # To Asset
# desc = f'bssimrun{yr}_{sim_date.strftime("%Y%m%d")}_{data_origin}_{cur_dt.strftime("%Y%m%d")}'

# utils.exportImgtoAsset(bs_composite, 
#                     desc=desc,
#                     region=None,
#                     asset_folder=asset_folder, 
#                     export_type='conus',
#                     export=True)
#                     #default=sensor) # TODO check if need to add sensor here, for NICF vs MTBS


SyntaxError: unmatched ')' (1451259440.py, line 104)

# For one fire 
### Select a fire from a pre-existing fire featurecollection or provide your own fire feature asset (must have 'Discovery' property with value of format ee.String('yyyy-mm-dd') )


In [2]:
# Running for a particular fire

# Select the fire based on irwinID (or name and edit filter below)
#fire_name = 'KNP COMPLEX' 
#fire_irwin = '499D6545-2A19-458A-8C7C-554932C6B86E'
#fire_name = 'WINDY' #there are 2 WINDY fires, using irwinID # fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'
fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'

# Read in fire data
fires = ee.FeatureCollection('projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2021_20230810')
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')
#print(fires.first().getInfo()['properties'])
#print(fires.aggregate_array('Incid_Name').distinct().sort().getInfo())

# Filter to desired fire
#fires_f = fires.filter(ee.Filter.eq('Incid_Name',fire_name))
fires_f = fires.filter(ee.Filter.eq('irwinID',fire_irwin))
print(f'Filtered fire size: {fires_f.size().getInfo()}')

fire_name = fires_f.first().get('Incid_Name').getInfo().lower() #brittle, make sure only 1 feature
print(fire_name)

# Run burn severity tool 
bs_coll = ee.FeatureCollection(fires_f).map(af.bs_calc_new)
#bs_fire = af.bs_calc_new(fires_f.first()) #testing direct function call
#print(bs_fire.propertyNames().getInfo())
# while max() isn't really needed for 1 fire, keeping the pattern of the multi-fire runs
bs_composite = ee.ImageCollection(bs_coll).max().add(1).rename('SEVERITY') #shift severity values +1 so 0 can be nodata

# To Asset
# get current date to put into asset name
cur_date = datetime.datetime.now().strftime("%Y%m%d")
desc = f'mtbs_bs_{str(fire_name).replace(" ", "")}_{str(cur_date)}'
print(desc)
#fails with Image.clipToBoundsAndScale: The geometry for image clipping must be bounded.
utils.exportImgtoAsset(bs_composite,  
                    desc=desc,
                    region=bs_coll.geometry(),
                    asset_folder='projects/pyregence-ee/assets/fires_bs_tool/dmn_testing', 
                    export_type='single_fire', 
                    export=False) 


Total Fires in FeatureCollection: 1022
Filtered fire size: 1
windy


## RDNBR calculation only, as a test

In [8]:
# Running for a particular fire - ONLY CALC RDNBR

# Select the fire based on irwinID 
    #fire_name = 'KNP COMPLEX' 
#fire_irwin = '499D6545-2A19-458A-8C7C-554932C6B86E'
    #fire_name = 'WINDY' #there are 2 WINDY fires, using irwinID # fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'
fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'

# Read in fire data
fires = ee.FeatureCollection('projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2021_20230810')
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')
#print(fires.first().getInfo()['properties'])
#print(fires.aggregate_array('Incid_Name').distinct().sort().getInfo())

# Filter to desired fire
#fires_f = fires.filter(ee.Filter.eq('Incid_Name',fire_name))
fires_f = fires.filter(ee.Filter.eq('irwinID',fire_irwin))
print(f'Filtered fire size: {fires_f.size().getInfo()}')

fire_name = fires_f.first().get('Incid_Name').getInfo().lower() #brittle, make sure only 1 feature
print(fire_name)

# CalC RDNBR
rdnbr_coll = ee.FeatureCollection(fires_f).map(af.rdnbr_only_calc)
#bs_fire = af.bs_calc_new(fires_f.first()) #testing direct function call
#print(bs_fire.propertyNames().getInfo())
# while max() isn't really needed for 1 fire, keeping the pattern of the multi-fire runs
rdnbr_composite = ee.ImageCollection(rdnbr_coll).max().rename('RDNBR') 

# To Asset
# get current date to put into asset name
cur_date = datetime.datetime.now().strftime("%Y%m%d")
desc = f'mtbs_bs_rdnbr_{str(fire_name).replace(" ", "")}_{str(cur_date)}'
print(desc)
#fails with Image.clipToBoundsAndScale: The geometry for image clipping must be bounded.
utils.exportImgtoAsset(rdnbr_composite,  
                    desc=desc,
                    region=rdnbr_coll.geometry(),
                    asset_folder='projects/pyregence-ee/assets/fires_bs_tool/dmn_testing', 
                    export_type='single_fire', 
                    export=False)


Total Fires in FeatureCollection: 1022
Filtered fire size: 1
knp complex
mtbs_bs_rdnbr_knpcomplex_20230829
export task started: projects/pyregence-ee/assets/fires_bs_tool/dmn_testing/mtbs_bs_rdnbr_knpcomplex_20230829


# ee authentication block

In [ ]:
# import ee
# ee.Authenticate()